In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import copy
from torchvision import models
import torch.nn.functional as F
from sklearn import metrics


In [4]:
#Define the model in pytorch using resnet32
class CNN_CXR(nn.Module):
    def __init__(self):
        super(CNN_CXR, self).__init__()
        ##import resnet
        self.model = torchvision.models.resnet34(pretrained=True)
        ##create new binary classifier (normal vs. pneumonia)
        classifier = nn.Sequential(
            nn.Linear(self.model.fc.in_features,128),
            nn.Linear(128,2),
            nn.LogSoftmax(dim =1))
        #Allow all parameters to be updated using the new model
        for params in self.model.parameters():
            params.requires_grad = True
        #Replace the output function with this classifier
        self.model.fc = classifier
    #Retain the same forward pass architecture in ResNet    
    def forward(self,x):
        return self.model(x)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Processor: {}'.format(device))

#Transform the images 
mean_vals = 0.5330
std_vals=0.0349

torch.manual_seed(0)
transformers = {'train_transforms' : transforms.Compose([
    transforms.Resize((224,224)),
    transforms.CenterCrop(224),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean_vals, std=std_vals)
]),
'test_transforms' : transforms.Compose([
    transforms.Resize((224,224)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean_vals, std=std_vals)
])}


#Load the images (batch loading)
trans = ['train_transforms', 'test_transforms']
path = "/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/chest_xray/"
categories = ['train_full','test']
dset = {x : torchvision.datasets.ImageFolder(path+x, transform=transformers[y]) for x,y in zip(categories, trans)}
dataset_sizes = {x : len(dset[x]) for x in ["train_full","test"]}
dataloaders =  {x : torch.utils.data.DataLoader(dset[x], batch_size=128, shuffle=True) for x in categories}
#load the model
model = CNN_CXR()
#loss function is NLL as can have class weights
weights = [0.7, 1]
class_weights=torch.FloatTensor(weights).cuda()
criterion  = nn.CrossEntropyLoss(weight=class_weights)
#optimizer is ADAM using standard parameters (can be tuned with cross-validation)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, weight_decay = 0.005)
# # of steps to take
total_step = dataset_sizes['train_full']

#create dictionaries for accuracy, probabiliteis and labels
train_accuracy = {}
train_probabilites = {}
train_labels = {}

test_accuracy = {}
test_probabilites = {}
test_labels = {}
num_epochs = 40

#copy the initial weights of the model
best_model_wts = copy.deepcopy(model.state_dict())
best_model_params = copy.deepcopy(optimizer.state_dict())
best_auc = 0.0
#run model
for epoch in range(num_epochs):
    #train the model on the train set
    correct_train = 0
    total_images_train = 0
    train_images_probabilites = []
    train_images_labels = []
    for i, (images, labels) in enumerate(dataloaders['train_full']):
        #move to cpu or gpu depending on processing unit
        model = model.to(device)
        images = images.to(device)
        labels = labels.to(device)
        #run model
        outputs = model(images)
        #wrangle output and labels and apply loss function
        labels = labels.type(torch.cuda.LongTensor)
        loss = criterion(outputs, labels)
        #store the model probabilites for being in class 1 for AUC
        prob = np.exp(outputs[:, 1].tolist())
        train_images_probabilites.extend(prob)
        train_images_labels.extend(labels.tolist())
        #assess accuracy of the train set
        predicted = np.round(prob)
        correct_train += np.where(predicted.astype(int) == np.array(labels.tolist()))[0].shape[0]
        total_images_train += labels.size(0)
        #apply backward propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_accuracy[epoch] = 100*(correct_train/total_images_train)
    train_probabilites[epoch] = train_images_probabilites
    train_labels[epoch] = train_images_labels
    
    fpr, tpr, thresholds = metrics.roc_curve(train_labels[epoch], train_probabilites[epoch])
    #auc
    print('Train: {}'.format(metrics.auc(fpr, tpr)))
    
    #Check accuracy in the test set
    with torch.no_grad():
        correct_test = 0
        total_images_test = 0
        test_images_probabilites = []
        test_images_labels = []



        for images, labels in dataloaders['test']:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)


            labels = labels.type(torch.cuda.LongTensor)
            loss = criterion(outputs, labels)

            prob = np.exp(outputs[:, 1].tolist())
            test_images_probabilites.extend(prob)
            test_images_labels.extend(labels.tolist())
            ##assess accuracy
            predicted = np.round(prob)
            correct_test += np.where(predicted.astype(int) == np.array(labels.tolist()))[0].shape[0]
            total_images_test += labels.size(0)

        test_accuracy[epoch] = 100*(correct_test/total_images_test)
        test_probabilites[epoch] = test_images_probabilites
        test_labels[epoch] = test_images_labels
        #calculate AUC for best epoch
        fpr, tpr, thresholds = metrics.roc_curve(test_labels[epoch], test_probabilites[epoch])
        AUC = metrics.auc(fpr, tpr)
        print('Test: {}'.format(AUC))
        
        #ensure we keep the best performing model in the test set for future use
        if AUC > best_auc:
            best_auc = AUC
            best_model_wts = copy.deepcopy(model.state_dict())
            best_opt_params = copy.deepcopy(optimizer.state_dict())
            ## save best model after each epoch as takes very long
            path = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/chest_xray/best_model_params.pt'
            state = { 'epoch': epoch,
            'state_dict': best_model_wts,
            'optimizer': best_opt_params }

            torch.save(state, path)
        

#auc
print('AUC: {}'.format(best_auc))




Processor: cuda
Train: 0.8678510086243174
Test: 0.8218833588449095
Train: 0.8780152980305048
Test: 0.8071062208296019
Train: 0.8836243258090484
Test: 0.8231421116424973
Train: 0.8887359195558726
Test: 0.817628649577206
Train: 0.8885654994018127
Test: 0.823646965025586
Train: 0.8928632385406629
Test: 0.8208350938395585
Train: 0.8955387196695319
Test: 0.8226867273655853
Train: 0.8993286194478778
Train: 0.9013456491991146
Test: 0.8329973258205383
Train: 0.9051396258947205
Test: 0.8512710671877521
Train: 0.9063272619680997
Test: 0.8361047586432414
Train: 0.9034935371337487
Test: 0.8294722777325803
Train: 0.9074719442093254
Test: 0.8423787268523434
Train: 0.9090661358186221
Test: 0.850288080248003
Train: 0.9100550723016213
Test: 0.8545837561192593
Train: 0.9098306664112245
Test: 0.8544247877974686
Train: 0.9111076771994492
Test: 0.8374041363170039
Train: 0.9101311803266954
Test: 0.8570931741664842
Train: 0.9112470860484516


In [87]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Processor: {}'.format(device))

#Transform the images 
mean_vals = 0.5330
std_vals=0.0349

torch.manual_seed(0)
transformers = {'train_transforms' : transforms.Compose([
    transforms.Resize((224,224)),
    transforms.CenterCrop(224),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean_vals, std=std_vals)
])}


#Load the images (batch loading)
trans = ['train_transforms']
path = "/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/small/cxr/"
categories = ['train']
dset = {x : torchvision.datasets.ImageFolder(path+x, transform=transformers[y]) for x,y in zip(categories, trans)}
dataset_sizes = {x : len(dset[x]) for x in ["train"]}
dataloaders =  {x : torch.utils.data.DataLoader(dset[x], batch_size=128, shuffle=True) for x in categories}


##load best model and optimizer
#path = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/chest_xray/best_model_params.pt'
#state = torch.load(path)
model = CNN_CXR()
#model.load_state_dict(state['state_dict'])
#optimizer.load_state_dict(state['optimizer'])
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, weight_decay = 0.005)

#loss function is NLL (allows weights per class)
weights = [0.2, 1]
class_weights=torch.FloatTensor(weights).cuda()
criterion  = nn.CrossEntropyLoss(weight=class_weights)


# # of steps to take
total_step = dataset_sizes['train']

#create dictionaries for accuracy, probabiliteis and labels
train_accuracy = {}
train_probabilites = {}
train_labels = {}


#copy the initial weights of the model
best_model_wts = copy.deepcopy(model.state_dict())
best_auc = 0.0

#run model
num_epochs = 20
for epoch in range(num_epochs):
    #train the model on the train set
    correct_train = 0
    total_images_train = 0
    train_images_probabilites = []
    train_images_labels = []
    for i, (images, labels) in enumerate(dataloaders['train']):
        #move to cpu or gpu depending on processing unit
        model = model.to(device)
        images = images.to(device)
        labels = labels.to(device)
        #run model
        outputs = model(images)
        #wrangle output and labels and apply loss function
        labels = labels.type(torch.cuda.LongTensor)
        loss = criterion(outputs, labels)
        #store the model probabilites for being in class 1 for AUC
        prob = np.exp(outputs[:, 1].tolist())
        train_images_probabilites.extend(prob)
        train_images_labels.extend(labels.tolist())
        #assess accuracy of the train set
        predicted = np.round(prob)
        correct_train += np.where(predicted.astype(int) == np.array(labels.tolist()))[0].shape[0]
        total_images_train += labels.size(0)
        #apply backward propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_accuracy[epoch] = 100*(correct_train/total_images_train)
    train_probabilites[epoch] = train_images_probabilites
    train_labels[epoch] = train_images_labels

    print('Train: {}'.format(train_accuracy[epoch]))
    
    #calculate AUC for best epoch
    fpr, tpr, thresholds = metrics.roc_curve(train_labels[epoch], train_probabilites[epoch])
    AUC = metrics.auc(fpr, tpr)
            
    #ensure we keep the best performing model in the test set for future use
    if AUC > best_auc:
        best_auc = AUC
        best_model_wts = copy.deepcopy(model.state_dict())
        best_opt_params = copy.deepcopy(optimizer.state_dict())

    #auc
    print('AUC: {}'.format(metrics.auc(fpr, tpr)))

print('Best AUC: {}'.format(best_acc))


Processor: cuda
Train: 74.771753307248


NameError: name 'best_auc' is not defined